In [4]:
import pandas as pd
import os

In [5]:
Fname="institution.csv"
Fdir="E:\PYTHON_work_learn\Python_work\Anu_Project\\from_Dropbox\\"
FnameFull=Fdir+Fname

In [6]:
FnameFull

'E:\\PYTHON_work_learn\\Python_work\\Anu_Project\\from_Dropbox\\institution.csv'

In [7]:
df=pd.read_csv(FnameFull,header=None)

In [8]:
df.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,AWI@FRIGOROPA,Polarstern core repository,27572 Bremerhaven,hannes.grobe@awi.de,Germany,+49 471 4831 1149,+49 471 4831 1220,NaN,Kühlhausstrasse 9,http://www.awi-bremerhaven.de/GEO/CoreRepository/,NaN,NaN,7.0
1,4,GeoB,"Department of Geosciences, Bremen University",28359 Bremen,NaN,Germany,+49 421 218 3116,+49 421 218 3585,NaN,Klagenfurter Strasse,http://www.geo.uni-bremen.de/page.php?langid=EN,NaN,NaN,7.0


In [9]:
colnamesstr="id_institution	abbreviation	name	city	contact	country	fax	phone	state	street	uri	uri_semantic	id_institution_type	id_user_created"

In [11]:
colnames=colnamesstr.split("\t")

In [177]:
df.columns=colnames
df.head()

,id_institution,abbreviation,name,city,contact,country,fax,phone,state,street,uri,uri_semantic,id_institution_type,id_user_created
0,1,AWI@FRIGOROPA,Polarstern core repository,27572 Bremerhaven,hannes.grobe@awi.de,Germany,+49 471 4831 1149,+49 471 4831 1220,NaN,Kühlhausstrasse 9,http://www.awi-bremerhaven.de/GEO/CoreRepository/,NaN,NaN,7.0
1,4,GeoB,"Department of Geosciences, Bremen University",28359 Bremen,NaN,Germany,+49 421 218 3116,+49 421 218 3585,NaN,Klagenfurter Strasse,http://www.geo.uni-bremen.de/page.php?langid=EN,NaN,NaN,7.0
2,5,GEOOSLO,"Department of Geosciences, University of Oslo",N-0316 Oslo 3,NaN,Norway,+47 22 85 4215,+47 22 85 6656,NaN,P.O. Box 1047 Blindern,http://www.geo.uio.no/english/,NaN,NaN,7.0
3,6,IFM-GEOMAR,IFM-GEOMAR Leibniz-Institute of Marine Science...,24148 Kiel,wbrueckmann@ifm-geomar.de,Germany,+49 431 600 2941,+49 431 600 2500 Sec,NaN,Wischhofstrasse 1-3,http://www.geomar.de,NaN,NaN,7.0
4,8,IMGP,"Göttinger Zentrum für Geowissenschaften, Unive...",37077 Göttingen,NaN,Germany,+49 551 397 996,+49 551 397 900,NaN,Goldschmidt-Strasse 3,http://www.imgp.gwdg.de/,NaN,NaN,7.0


### Time to make some requests 

In [15]:
import requests

## Query functions

In [137]:
def getresponse(url,params):
    try:
        r=requests.get(url,params=params)
        if r.status_code!=200:
            raise requests.HTTPError
        return r
    except requests.exceptions.HTTPError:
        return("Http error!")
    except requests.exceptions.ConnectionError:
        return("Connection Error!")
    except requests.exceptions.Timeout:
        return("Timeout error!")
    except requests.exceptions.RequestException:
        return("Unknown  Requests error occured!") 

In [138]:
def rparser(response):
    try:
        if type(response)!=requests.models.Response:
            raise TypeError
        jout=response.json()
        if jout["number_of_results"]!=0:
            return jout['items']
        else:
            print("empty response!")
            return None 
    except TypeError:
        return "wrong input type!"
    except KeyError:
        return "KeyError - query API error"

## Query example (Bremen University)

Query parameters

In [18]:
name=df["name"][1]
name

'Department of Geosciences, Bremen University'

In [23]:
country=df["country"][1]
country

'Germany'

In [24]:
API_url="https://api.ror.org/organizations"
params={"query":[name],"filter":"country.country_name:"+country}

Query itself

In [39]:
r=getresponse(API_url,params)
r.text[0:400]

'{"number_of_results":537,"time_taken":12,"items":[{"id":"https://ror.org/04ers2y35","name":"University of Bremen","types":["Education"],"links":["http://www.uni-bremen.de/en.html"],"aliases":[],"acronyms":[],"wikipedia_url":"http://en.wikipedia.org/wiki/University_of_Bremen","labels":[{"label":"Universität Bremen","iso639":"de"}],"country":{"country_name":"Germany","country_code":"DE"},"external_i'

In [40]:
r.url

'https://api.ror.org/organizations?query=Department+of+Geosciences%2C+Bremen+University&filter=country.country_name%3AGermany'

Fisrt 20 results. We will pick the first one and take its ROR_id. This is not going to work in some cases!

In [41]:
jdic=rparser(r)
[s["name"] for s in jdic]

['University of Bremen',
 'Jacobs University',
 'Bremen University of Applied Sciences',
 'University of the Arts Bremen',
 'Universum Bremen',
 'Staatsarchiv Bremen',
 'Freie Universität Berlin',
 'European Geosciences Union',
 'Hochschule für Technik und Wirtschaft des Saarlandes',
 'Hessische Hochschule für Polizei und Verwaltung',
 'Hochschule Bremerhaven',
 'Hochschule Mittweida',
 'Klinikum Bremen-Mitte',
 'Hochschule Osnabrück',
 'Technische Hochschule Mittelhessen',
 'Institute for Information Management Bremen',
 'Hochschule Ulm',
 'Fliedner Fachhochschule Düsseldorf',
 'Hochschule für bildende Künste',
 'Hochschule Magdeburg-Stendal']

In [42]:
print(jdic[0]["id"])

https://ror.org/04ers2y35


## Query fill

lets create a list of ROR_id's received by taking the first element of a query result

In [105]:
qror_list=[None]*len(df) # creating an empty list of fixed size - of length of database entries
API_url="https://api.ror.org/organizations"

In [108]:
name=df["country"][9]
name

nan

In [183]:
def idreq(i):
    try:
        name=df["name"][i]
        country=df["country"][i]
        params={"query":[name],"filter":"country.country_name:"+country}
    except TypeError:
        return name,"TypeError occured while reading name or country"
    r=getresponse(API_url,params)
    jdic=rparser(r)
    if type(jdic)==list:
        return name,jdic[0]["id"]  # return the ROR id of the FIRST object of JSON
    else:
        return name,jdic

In [192]:
for s in range(999,len(df)+1):
    name,val=idreq(s)
    print("Iteration {s}, output === {p} // name==={n} \n #############".format(s=s,p=val,n=name))
    qror_list[s]=val

Iteration 999, output === https://ror.org/01xtthb56 // name===Section of Marine Botany, University of Oslo 
 #############
Iteration 1000, output === https://ror.org/01xtthb56 // name===Section of Marine Zoology and Marine Chemistry, University of Oslo 
 #############
Iteration 1001, output === https://ror.org/01xtthb56 // name===Department of Geophysics, University of Oslo 
 #############
Iteration 1002, output === https://ror.org/01xtthb56 // name===Department of Mathematics, University of Oslo 
 #############
Iteration 1003, output === https://ror.org/022bzg803 // name===Department of Pharmacology, Microbiology and Food Hygiene, Norwgian Centre of Ve 
 #############
Iteration 1004, output === https://ror.org/00by76g53 // name===Department of Fish Health, Central Veterinary Laboratory 
 #############
Iteration 1005, output === https://ror.org/0502t5s28 // name===RF-Rogaland Research 
 #############
Iteration 1006, output === https://ror.org/05dz27378 // name===Norwegian Hydrographic 

Iteration 1060, output === https://ror.org/02y9x6z24 // name===Centro de Oceanografia, Departmento de Fisica, Centro de Geofisica, Universidade 
 #############
Iteration 1061, output === https://ror.org/01sp7nd78 // name===Departmento de Zoologia, Instituto do Mar 
 #############
Iteration 1062, output === https://ror.org/014837179 // name===Instituto do Mar, Faculdade de Ciencias e Tecnologicia, Universidade Nova de Lis 
 #############
Iteration 1063, output === https://ror.org/007qsya74 // name===Instituto Portuges de Investigaçao Maritima, C. Reg. de Invest. Maritima-Sul 
 #############
Iteration 1064, output === https://ror.org/05v384f19 // name===Instituto do Mar, Universidade do Porto 
 #############
Iteration 1065, output === https://ror.org/04ehtgm24 // name===Instituto de Ciéncias Biomédicas Abel Salazar, Universidade do Porto 
 #############
Iteration 1066, output === https://ror.org/04ehtgm24 // name===Faculdade de Ciencias, Universidade do Porto 
 #############
Iteration 10

Iteration 1119, output === https://ror.org/040af2s02 // name===Tväminne Zoological Station, University of Helsinki 
 #############
Iteration 1120, output === https://ror.org/05hppb561 // name===Headquartetrs and Aquaculture & Fisheries Division, Finnish Institute of Game an 
 #############
Iteration 1121, output === https://ror.org/013nat269 // name===Finnish Environment Institute 
 #############
Iteration 1122, output === https://ror.org/03k90y290 // name===Finnish Institute of Marine Research, Helsinki 
 #############
Iteration 1123, output === https://ror.org/03vjnqy43 // name===Hydrographic Survey Division 
 #############
Iteration 1124, output === https://ror.org/036bxpj43 // name===Department of Limnology and Environmental Protection Section of Limnology, Unive 
 #############
Iteration 1125, output === https://ror.org/01j45tp19 // name===Karelian Institute Section of Ecology, University of Joensuu 
 #############
Iteration 1126, output === https://ror.org/01zv3gf04 // name===Åla

Iteration 1180, output === https://ror.org/03sbpja79 // name===Department of Applied Mathematics and Theoretical Physics, University of Cambrid 
 #############
Iteration 1181, output === https://ror.org/01zewfb16 // name===H.Q. Fisheries and Conservation Department, Environment Agency Welsh Region 
 #############
Iteration 1182, output === https://ror.org/01se4f844 // name===Department of Maritime Studies, University of Wales, Institute of Science & Tech 
 #############
Iteration 1183, output === wrong input type! // name===Fisheries Research Laboratory, Department of Agriculture for Northern Ireland (D 
 #############
Iteration 1184, output === https://ror.org/0021w4337 // name===Fisheries Laboratory, Ministry of Agriculture, Fisheries and Food, Conwy 
 #############
Iteration 1185, output === https://ror.org/043j90n04 // name===RVAS Aquatic Resources Unit 
 #############
Iteration 1186, output === https://ror.org/03h2bxq36 // name===Department of Civil Engineering, University of Dund

Iteration 1240, output === https://ror.org/01vhnrs90 // name===Max-Planck-Institut für Astronomie 
 #############
Iteration 1241, output === https://ror.org/0243gzr89 // name===Max-Planck-Institut für Biologie, Tübingen 
 #############
Iteration 1242, output === https://ror.org/022jc0g24 // name===Max-Planck-Institut für Entwicklungsbiologie 
 #############
Iteration 1243, output === https://ror.org/005bk2339 // name===Max-Planck-Institut für Festkörperforschung 
 #############
Iteration 1244, output === https://ror.org/04vh1tq58 // name===Friedrich-Miescher-Laboratorium für biologische Arbeitsgruppen in der Max-Planck 
 #############
Iteration 1245, output === https://ror.org/03fkg8131 // name===Max-Planck-Institut für Immunbiologie 
 #############
Iteration 1246, output === https://ror.org/052d0h423 // name===Max-Planck-Institut für Kernphysik 
 #############
Iteration 1247, output === https://ror.org/026nmvv73 // name===Max-Planck-Institut für Kybernetik 
 #############
Iteration 12

...

Iteration 9147, output === https://ror.org/01nrxwf90 // name===School of Physics and Astronomy, University of Edinburgh 
 #############
Iteration 9148, output === https://ror.org/02n415q13 // name===Department of Chemistry, Faculty of Science and Engeneering, Curtin University 
 #############
Iteration 9149, output === https://ror.org/01dq60k83 // name===International Research Institute of Disaster Science, Tohoku University 
 #############
Iteration 9150, output === https://ror.org/01r4eh644 // name===Lunar and Planetary Institute, Universities Space Research Association 
 #############
Iteration 9151, output === https://ror.org/04gcegc37 // name===School of Earth Sciences, Planetary Science Institute, China University of Geosciences (Wuhan) 
 #############
Iteration 9152, output === https://ror.org/014net767 // name===National Center of Scientific Research (CNRS), Groupe de Physico-Chimie de l'Atmosphère, Institut de Chimie et Procédés pour l'Energie, l'Environnement et la Santé ICPE

Iteration 9206, output === https://ror.org/056tb7j80 // name===Universidad Nacional de Córdoba 
 #############
Iteration 9207, output === TypeError occured while reading name or country // name===Geophysical Institute, University of Bergen and Bjerknes Centre for Climate Research 
 #############
Iteration 9208, output === https://ror.org/00ctdh943 // name===Los Alamos National Laboratory 
 #############
Iteration 9209, output === https://ror.org/046ak2485 // name===Institut für Meteorologie, Freie Universität Berlin 
 #############
Iteration 9210, output === TypeError occured while reading name or country // name===Marine Institute, University of Texas, Galvestone 
 #############
Iteration 9211, output === TypeError occured while reading name or country // name===El Mercurio Peruano (newspaper) 
 #############
Iteration 9212, output === https://ror.org/02ttsq026 // name===Colorado Center for Astrodynamics Research, University of Colorado Boulder 
 #############
Iteration 9213, output =

Iteration 9268, output === https://ror.org/03mb6wj31 // name===Universitat Politècnica de Catalunya 
 #############
Iteration 9269, output === https://ror.org/02s376052 // name===École Polytechnique Fédérale de Lausanne 
 #############
Iteration 9270, output === https://ror.org/03cqe8w59 // name===National Scientific and Technical Research Council 
 #############
Iteration 9271, output === https://ror.org/014g34x36 // name===Universidade do Algarve 
 #############
Iteration 9272, output === TypeError occured while reading name or country // name===Department of Biosciences, Aarhus University, and Institute for Hydrobiology and Fisheries Science, University of Hamburg 
 #############
Iteration 9273, output === https://ror.org/047272k79 // name===The University of Western Australia 
 #############
Iteration 9274, output === TypeError occured while reading name or country // name===COSYNA - Helmholtz-Zentrum Geesthacht Zentrum für Material- und Küstenforschung GmbH 
 #############
Iterati

Iteration 9331, output === https://ror.org/0040zx077 // name===Istituto di ricerca per la Protezione Idrogeologica del Consiplio Nazionale delle Ricerche 
 #############
Iteration 9332, output === https://ror.org/01jr63883 // name===Geographisches Institut, Universität Zürich 
 #############
Iteration 9333, output === https://ror.org/03efzvv60 // name===IADO-CONICET Bahía Blanca 
 #############
Iteration 9334, output === https://ror.org/013meh722 // name===Department of Geography, University of Cambridge 
 #############
Iteration 9335, output === TypeError occured while reading name or country // name===Deutsche Seewarte 
 #############
Iteration 9336, output === TypeError occured while reading name or country // name===Laboratorio di Urbanistica e di Pianificazione Territoriale - Raffaele d'Ambrosio 
 #############
Iteration 9337, output === TypeError occured while reading name or country // name===Brookhaven National Laboratory 
 #############
Iteration 9338, output === TypeError occ

KeyError: 9376

In [195]:
len(qror_list)

9376

In [199]:
qror_list[0:10]

[None,
 'https://ror.org/04ers2y35',
 'https://ror.org/01xtthb56',
 'https://ror.org/02h2x0161',
 'https://ror.org/01y9bpm73',
 'https://ror.org/04v76ef78',
 'https://ror.org/01se4f844',
 None,
 'https://ror.org/02dvf9b44',
 'TypeError occured while reading name or country']

Lets insert new ROR_id column!

In [201]:
df.insert(0,"qROR_id",qror_list)

In [202]:
df

,qROR_id,id_institution,abbreviation,name,city,contact,country,fax,phone,state,street,uri,uri_semantic,id_institution_type,id_user_created
0,None,1,AWI@FRIGOROPA,Polarstern core repository,27572 Bremerhaven,hannes.grobe@awi.de,Germany,+49 471 4831 1149,+49 471 4831 1220,NaN,Kühlhausstrasse 9,http://www.awi-bremerhaven.de/GEO/CoreRepository/,NaN,NaN,7.0
1,https://ror.org/04ers2y35,4,GeoB,"Department of Geosciences, Bremen University",28359 Bremen,NaN,Germany,+49 421 218 3116,+49 421 218 3585,NaN,Klagenfurter Strasse,http://www.geo.uni-bremen.de/page.php?langid=EN,NaN,NaN,7.0
2,https://ror.org/01xtthb56,5,GEOOSLO,"Department of Geosciences, University of Oslo",N-0316 Oslo 3,NaN,Norway,+47 22 85 4215,+47 22 85 6656,NaN,P.O. Box 1047 Blindern,http://www.geo.uio.no/english/,NaN,NaN,7.0
3,https://ror.org/02h2x0161,6,IFM-GEOMAR,IFM-GEOMAR Leibniz-Institute of Marine Science...,24148 Kiel,wbrueckmann@ifm-geomar.de,Germany,+49 431 600 2941,+49 431 600 2500 Sec,NaN,Wischhofstrasse 1-3,http://www.geomar.de,NaN,NaN,7.0
4,https://ror.org/01y9bpm73,8,IMGP,"Göttinger Zentrum für Geowissenschaften, Unive...",37077 Göttingen,NaN,Germany,+49 551 397 996,+49 551 397 900,NaN,Goldschmidt-Strasse 3,http://www.imgp.gwdg.de/,NaN,NaN,7.0
5,https://ror.org/04v76ef78,9,GIK,"Geologisch-Paläontologisches Institut, Christi...",24098 Kiel,name changed to Inst. für Geowissenschaften,Germany,NaN,NaN,NaN,NaN,http://www.ifg.uni-kiel.de/,NaN,0.0,7.0
6,https://ror.org/01se4f844,11,IES,"Institute of Geography and Earth Sciences, Uni...","Aberystwyth, SY23 3DB, Wales",glawww@aber.ac.uk,United Kingdom,+44 1970 622 780,+44 1970 622 781,NaN,Ceredigion,http://www.aber.ac.uk/~gbb/chome.html,NaN,NaN,7.0
7,None,13,RGD,"Rijks Geologische Dienst, Marine Geology Depar...",2000 AD Haarlem,NaN,The Netherlands,NaN,023-319362,NaN,P.O. Box 157,NaN,NaN,NaN,7.0
8,https://ror.org/02dvf9b44,15,IUPH,"Institut für Umweltphysik, Heidelberger Akadem...",69120 Heidelberg,Bernd Kromer,Germany,+49 6221 546 405,+49 6221 546 357,NaN,Im Neuenheimer Feld 366,http://www.uphys.uni-heidelberg.de/,NaN,NaN,7.0
9,TypeError occured while reading name or country,17,n_g,not_given,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0


In [219]:
df.to_csv("test.csv",sep=",")

In [216]:
path=os.chdir("E:\PYTHON_work_learn\Python_work\Anu_Project")

In [217]:
os.getcwd()

'E:\\PYTHON_work_learn\\Python_work\\Anu_Project'